In [ ]:
# YOLOE-11L (seg checkpoint)
!wget https://github.com/ultralytics/assets/releases/download/v8.3.0/yoloe-11l-seg.pt

In [6]:
!pip install -U ultralytics onnx onnxruntime onnxslim

  Using cached ultralytics-8.3.250-py3-none-any.whl.metadata (37 kB)
  Using cached onnx-1.20.0-cp312-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.4 kB)
  Using cached onnxruntime-1.23.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.1 kB)
  Using cached onnxslim-0.1.82-py3-none-any.whl.metadata (10 kB)
  Using cached matplotlib-3.10.8-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (52 kB)
  Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchvision-0.24.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached ml_dtypes-0.5.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cac

Ultralytics points to the RAM++ predefined list

In [9]:
!curl -L -o labels_yoloe_pf_4585.txt \
  https://github.com/xinyu1205/recognize-anything/raw/main/ram/data/ram_tag_list.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 41904  100 41904    0     0  61843      0 --:--:-- --:--:-- --:--:--  335k


#### Convert the segmentation checkpoint to a detection checkpoint (optional but cleaner)

In [7]:
from ultralytics import YOLOE
from ultralytics.utils.patches import torch_load

# Build a detection model skeleton, then load seg weights into it
det_model = YOLOE("yoloe-11l.yaml")
state = torch_load("yoloe-11l-seg.pt")
det_model.load(state["model"])
det_model.save("yoloe-11l-det.pt")

print("Saved: yoloe-11l-det.pt")


WARNING ⚠️ user config directory '/root/.config/Ultralytics' is not writable, using '/tmp/Ultralytics'. Set YOLO_CONFIG_DIR to override.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/tmp/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Transferred 1108/1108 items from pretrained weights
Saved: yoloe-11l-det.pt


Freeze your “open vocab” class list + export ONNX

In [15]:
from ultralytics import YOLOE

# Load your detection checkpoint (or use yoloe-11l-seg.pt directly if you skip conversion)
model = YOLOE("yoloe-11l-det.pt")

# Choose the classes from YOLOE to detect in DeepStream
# names = ["person", "car", "bus", "truck", "motorcycle", "bicycle"]
# COCO80 = [
#   "person","bicycle","car","motorcycle","airplane","bus","train","truck","boat",
#   "traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat","dog","horse","sheep","cow",
#   "elephant","bear","zebra","giraffe","backpack","umbrella","handbag","tie","suitcase",
#   "frisbee","skis","snowboard","sports ball","kite","baseball bat","baseball glove","skateboard","surfboard","tennis racket",
#   "bottle","wine glass","cup","fork","knife","spoon","bowl",
#   "banana","apple","sandwich","orange","broccoli","carrot","hot dog","pizza","donut","cake",
#   "chair","couch","potted plant","bed","dining table","toilet","tv","laptop","mouse","remote","keyboard","cell phone",
#   "microwave","oven","toaster","sink","refrigerator","book","clock","vase","scissors","teddy bear","hair drier","toothbrush"
# ]

TRAFFIC_LABELS = [
  # People & PPE
  "person","worker","pedestrian","cyclist","motorcyclist",
  "helmet","hard hat","reflective vest","face mask","backpack",

  # Core vehicles
  "bicycle","motorcycle","scooter","pmv","pmd",
  "car","taxi","van","pickup truck","minibus","bus",
  "truck","box truck","flatbed truck","tanker truck","container truck","trailer",
  "tow truck","ambulance","fire truck","police car","maintenance vehicle",

  # Roadway furniture / signs / signals
  "traffic light","pedestrian signal","traffic sign","speed limit sign","no entry sign",
  "stop sign","yield sign","direction sign","gantry sign","variable message sign",
  "road cone","bollard","barrier","jersey barrier","guardrail","fence",
  "lane marking","zebra crossing","road shoulder","kerb",

  # Common carried / dropped objects
  "umbrella","luggage","suitcase","box","cart","stroller",

  # Hazards / events (as “objects” you may want to detect)
  "smoke","fire","debris","fallen tree","spill","puddle","flooding",
  "accident","crashed vehicle","stopped vehicle",

  # Animals (useful for expressways)
  "dog","cat","bird"
]



# Freeze the text prompts into the model before export (key for DeepStream)
model.set_classes(TRAFFIC_LABELS, model.get_text_pe(TRAFFIC_LABELS))

# Export ONNX (static batch is simplest for DeepStream)
export_path = model.export(
    format="onnx",
    imgsz=640,        # set to your inference size
    opset=12,         # safe default for compatibility
    simplify=True,
    dynamic=False
)

# Write DeepStream labels file (order must match the names (labels) you set above)
with open("labels_yoloe.txt", "w") as f:
    for n in TRAFFIC_LABELS:
        f.write(n + "\n")

print("Exported:", export_path)
print("Wrote: labels_yoloe.txt")


Ultralytics 8.3.250 🚀 Python-3.12.3 torch-2.9.1+cu128 CPU (Intel Core Ultra 7 265KF)
YOLOe-11l summary (fused): 215 layers, 32,812,582 parameters, 0 gradients, 88.8 GFLOPs

PyTorch: starting from 'yoloe-11l-det.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 75, 8400) (63.2 MB)

ONNX: starting export with onnx 1.20.0 opset 12...
ONNX: slimming with onnxslim 0.1.82...
ONNX: export success ✅ 1.3s, saved as 'yoloe-11l-det.onnx' (97.0 MB)

Export complete (1.7s)
Results saved to /opt/nvidia/deepstream/deepstream-8.0/samples/playground
Predict:         yolo predict task=detect model=yoloe-11l-det.onnx imgsz=640  
Validate:        yolo val task=detect model=yoloe-11l-det.onnx imgsz=640 data=None  
Visualize:       https://netron.app
Exported: yoloe-11l-det.onnx
Wrote: labels_yoloe.txt


In [16]:
from ultralytics import YOLO

# Load pretrained YOLOE model and train on custom data
model = YOLO("yoloe-11l-seg.pt")

# Run inference using text prompts ("person", "bus")
model.set_classes(TRAFFIC_LABELS)
results = model.predict(source="crowd.jpg")
results[0].save()  # save annotated output


image 1/1 /opt/nvidia/deepstream/deepstream-8.0/samples/playground/crowd.jpg: 448x640 90 persons, 1 backpack, 3 bicycles, 1 car, 1 traffic sign, 1 direction sign, 1 umbrella, 2 strollers, 132.0ms
Speed: 0.9ms preprocess, 132.0ms inference, 12.2ms postprocess per image at shape (1, 3, 448, 640)


'results_crowd.jpg'